# 1.0 INTRODUCTION

As more and more people become wealthy, the proportion of disposable income also increases and this in turn causes more people to eat meals not prepared in the home. Coupled with this, is the trend towards working more hours and importantly socializing outside of work and traditional family structures. Even though we have become more digital we have also become less glued to our home kitchen to provide that necessary nutrition. We are spending more and more time eating in restaurants and fast food outlets. Many companies are strategizing to leverage this opportunity and so is XYZ Ltd a successful UK based restaurant chain, headquartered in London, UK.

# 2.0 BACKGROUND

In 2018, Victor Oged was appointed CEO of XYZ Ltd the parent company of the NetherSpoons brand of restaurants. NeterSpoons has had a long heritage of serving traditional English breakfast, sizzling fish and chips and its flagship English pudding primarily around London. Mr Oged was appointed to reinvigorate this long standing British business, which has existed for over 100 years. With his success in increasing the revenues and locations of Pular Fashion Retail by 30 and 10 per cent respectively, Mr Oged has set a target of increasing NetherSpoons revenues by 15% by 2020. At 2018 XYZ AGM, Mr Oged stated he will driving the international expansion of the business.

# 3.0 THE PROBLEM

In order to achieve this objective of international expansion, Mr Oged has earmarked £100M for this, over the next 2 years. He charged the Investment Team to identify the first international location of NetherSpoons.

The investment team after much deliberation have settled on Canada and specifically Toronto because ofthe following:
 -  There is a long history between Canada and England, with similar heritages
 -  Both countries share a common language, ease of communication
 -  NetherSpoons major revenue location is London, with it's cosmopolitan atmosphere, Toronto appears similar in demographics to London
 - There are many English immigrants to Canada, who might be interested in a traditional English dishes served by NetherSpoons
 -  NetherSpoons uses traditional English decor in the interior setting, successfully in London West End, this might work also in Toronto
 
The business case for the first international location for NetherSpoons has been approved by Mr Oged and the board. In order to test the assumptions of the Investment Team, Mr Oged has asked DataScience Inc to analyse the neighbourhoods in London and Toronto and answer the following

- How similar are the types of Venues in London West End compared to Toronto?
- can we infer from the data that a NetherSpoons restaurant will be succesful
- Identify the best 3 locations in Toronto that is similar to London's West End to site the restaurant

They are to present their findings to the Investment Team, who will in turn make recommendations to Mr Oged, in 4 weeks time.

***

# 4.0 DATA

I will be working with 5 groups of data sources

1. Wikipedia page listing all London boroughs: This data source has both names of London's 33 boroughs and their grid references (https://en.wikipedia.org/wiki/List_of_London_boroughs). This will be sued to get the Longitude and Latitude of Westminster and Camden; West End boroughs.
2. Additional resource will be used to supplent the Wikipedia source. CSV file downloaded from https://www.doogal.co.uk/london_postcodes.php This file contains all postal codes within London. In order to compare favourably both locations in London and Toronto, we will use the postcode group.
3. Foursquare API to get list of Venues in both locations https://foursquare.com/
4. For Toronto we'll use Wikipedia to get Nieighbourhood names https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
5. GeoSpatial Data with Latitudes and Longitudes https://cocl.us/Geospatial_data



***

# 5.0 METHODOLOGY

This section of the notebook is broken down into:

<b>Toronto</b>
 - Data retrieval
 - Data cleaning
 - Data exploration
 - Clustering
 
<b>London</b>
 - Data retrieval
 - Data cleaning
 - Data exploration
 - Clustering
 

### <i>5.1 IMPORTING ALL NECESSARY MODULES</i>

In [1]:
# This cell imports all necessary Python modules needed for analysis
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

## 5.2 TORONTO
### DATA RETRIEVAL

In [2]:
#Wikipedia link to Toronto postalcodes and neighbourhoods
toronto_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [3]:
toronto_data=pd.read_html(toronto_link)

In [4]:
toronto_df= toronto_data[0]
toronto_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 288 entries, 0 to 287
Data columns (total 3 columns):
Postcode         288 non-null object
Borough          288 non-null object
Neighbourhood    288 non-null object
dtypes: object(3)
memory usage: 6.8+ KB


## TORONTO
### DATA CLEANING

In [5]:
col=['PostalCode','Borough','Neighborhood']
toronto_df.columns=col
toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [6]:
toronto_df= toronto_df.loc[toronto_df['Borough']!='Not assigned']
toronto_df= toronto_df.reset_index()
toronto_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211 entries, 0 to 210
Data columns (total 4 columns):
index           211 non-null int64
PostalCode      211 non-null object
Borough         211 non-null object
Neighborhood    211 non-null object
dtypes: int64(1), object(3)
memory usage: 6.7+ KB


In [7]:
print(toronto_df.loc[toronto_df['Neighborhood']=='Not assigned'])
toronto_df.at[6,'Neighborhood']= "Queen's Park"
toronto_df.iloc[6]

   index PostalCode       Borough  Neighborhood
6      8        M7A  Queen's Park  Not assigned


index                      8
PostalCode               M7A
Borough         Queen's Park
Neighborhood    Queen's Park
Name: 6, dtype: object

In [8]:
toronto_df= toronto_df.drop('index',axis=1)
toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [9]:
#Identifying neighbourhoods with the same postalcodes
toronto_duplicates= toronto_df.loc[toronto_df.duplicated('PostalCode',keep=False)]
print(toronto_duplicates.shape)
toronto_duplicates.head()

(165, 3)


,PostalCode,Borough,Neighborhood
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
8,M1B,Scarborough,Rouge


In [10]:
#Grouping similar postalcodes together
toronto_grp= toronto_duplicates.groupby(['PostalCode', 'Borough'], as_index=False).agg(lambda x: ", ".join(x))
print(toronto_grp.shape)
toronto_grp.head()

(57, 3)


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
4,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"


In [11]:
toronto_part= toronto_df.loc[~toronto_df.duplicated('PostalCode',keep=False)]
toronto_part.shape

(46, 3)

In [12]:
tor_df= pd.concat([toronto_part,toronto_grp],ignore_index=True)
tor_df.shape

(103, 3)

In [13]:
# Retrieving location data
toronto_geoRef= pd.read_csv('https://cocl.us/Geospatial_data')
print(toronto_geoRef.shape)
toronto_geoRef.head()

(103, 3)


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
toronto_geoRef.columns= ['PostalCode','Latitude','Longitude']
toronto_newDf= pd.merge(tor_df,toronto_geoRef, on='PostalCode')
print(toronto_newDf.shape)
toronto_newDf.head()

(103, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
3,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
4,M3B,North York,Don Mills North,43.745906,-79.352188


## TORONTO
### DATA EXPLORATION

In [15]:
# Mapping Toronto
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [16]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_newDf['Latitude'], toronto_newDf['Longitude'], toronto_newDf['Borough'], toronto_newDf['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### <i> UTILIZING THE FOURSUARE API</i>

In [17]:
# The code was removed by Watson Studio for sharing.

In [18]:
LIMIT=100 # setting limit returned by the Foursquare API

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [19]:
toronto_venues = getNearbyVenues(names=toronto_newDf['Neighborhood'],
                                   latitudes=toronto_newDf['Latitude'],
                                   longitudes=toronto_newDf['Longitude']
                                  )

Parkwoods
Victoria Village
Queen's Park
Islington Avenue
Don Mills North
Glencairn
Woodbine Heights
St. James Town
Humewood-Cedarvale
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Thorncliffe Park
Scarborough Village
East Toronto
Bayview Village
Downsview West
Humber Summit
Downsview Central
Studio District
Willowdale South
Downsview Northwest
Lawrence Park
Roselawn
Weston
York Mills West
Davisville North
Westmount
Willowdale West
North Toronto West
Canada Post Gateway Processing Centre
Agincourt
Davisville
L'Amoreaux West
Rosedale
Stn A PO Boxes 25 The Esplanade
Northwest
Upper Rouge
Church and Wellesley
Business Reply Mail Processing Centre 969 Eastern
Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Tow

In [20]:
#Shape of the resulting Dataframe
print(toronto_venues.shape)
toronto_venues.head()

(2283, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [21]:
#How many Venues were retured for each Neighbourhood
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,5,5,5,5,5,5
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",2,2,2,2,2,2
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",11,11,11,11,11,11
"Alderwood, Long Branch",11,11,11,11,11,11
"Bathurst Manor, Downsview North, Wilson Heights",19,19,19,19,19,19
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",21,21,21,21,21,21
Berczy Park,58,58,58,58,58,58


In [22]:
#How many unique Venue categories?
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 276 uniques categories.


## 5.3 TORONTO
### <i> NEIGHBOURHOOD ANALYSIS</i>

In [23]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [24]:
toronto_onehot.shape

(2283, 276)

In [25]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
print(toronto_grouped.shape)
toronto_grouped

(99, 276)


,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.000000,0.000000,0.00000

## 5.4 TORONTO

### <i>TOP 5 VENUES</i>

In [26]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0          Coffee Shop  0.08
1                 Café  0.05
2  American Restaurant  0.04
3                  Bar  0.04
4      Thai Restaurant  0.04


----Agincourt----
                venue  freq
0              Lounge   0.2
1      Breakfast Spot   0.2
2        Skating Rink   0.2
3      Sandwich Place   0.2
4  Chinese Restaurant   0.2


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                             venue  freq
0                       Playground   0.5
1                             Park   0.5
2                      Yoga Studio   0.0
3                      Men's Store   0.0
4  Molecular Gastronomy Restaurant   0.0


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                  venue  freq
0           Pizza Place  0.18
1         Grocery Store  0.18
2              Pharmacy  0.09
3            Beer Store  0.09
4  Fast Food Restauran

In [27]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [28]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Bar,Thai Restaurant,American Restaurant,Steakhouse,Asian Restaurant,Sushi Restaurant,Restaurant,Hotel
1,Agincourt,Lounge,Chinese Restaurant,Skating Rink,Breakfast Spot,Sandwich Place,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Playground,Women's Store,Doner Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Pizza Place,Japanese Restaurant,Fast Food Restaurant,Beer Store,Sandwich Place,Discount Store,Fried Chicken Joint,Pharmacy,Concert Hall
4,"Alderwood, Long Branch",Pizza Place,Gym,Sandwich Place,Dance Studio,Athletics & Sports,Pool,Pub,Pharmacy,Coffee Shop,Skating Rink


## 5.5 TORONTO
### CLUSTERING

In [29]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 2, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [30]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_newDf

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,1.0,Fast Food Restaurant,Park,Food & Drink Shop,Women's Store,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,Hockey Arena,Coffee Shop,French Restaurant,Portuguese Restaurant,Intersection,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Department Store
2,M7A,Queen's Park,Queen's Park,43.662301,-79.389494,0.0,Coffee Shop,Park,Gym,Diner,Seafood Restaurant,Sandwich Place,Salad Place,Burger Joint,Burrito Place,Café
3,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,M3B,North York,Don Mills North,43.745906,-79.352188,0.0,Gym / Fitness Center,Caribbean Restaurant,Basketball Court,Japanese Restaurant,Café,Construction & Landscaping,Diner,Event Space,Ethiopian Restaurant,Empanada Restaurant


In [31]:
toronto_merged.dropna(axis=0,inplace=True)
toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,1.0,Fast Food Restaurant,Park,Food & Drink Shop,Women's Store,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,Hockey Arena,Coffee Shop,French Restaurant,Portuguese Restaurant,Intersection,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Department Store
2,M7A,Queen's Park,Queen's Park,43.662301,-79.389494,0.0,Coffee Shop,Park,Gym,Diner,Seafood Restaurant,Sandwich Place,Salad Place,Burger Joint,Burrito Place,Café
4,M3B,North York,Don Mills North,43.745906,-79.352188,0.0,Gym / Fitness Center,Caribbean Restaurant,Basketball Court,Japanese Restaurant,Café,Construction & Landscaping,Diner,Event Space,Ethiopian Restaurant,Empanada Restaurant
5,M6B,North York,Glencairn,43.709577,-79.445073,0.0,Park,Pizza Place,Metro Station,Pub,Japanese Restaurant,Discount Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner


In [32]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels'].astype(int)):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5.6 TORONTO
### EXAMINING EACH CLUSTER

#### Cluster 1

In [33]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,0.0,Hockey Arena,Coffee Shop,French Restaurant,Portuguese Restaurant,Intersection,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Department Store
2,Queen's Park,0.0,Coffee Shop,Park,Gym,Diner,Seafood Restaurant,Sandwich Place,Salad Place,Burger Joint,Burrito Place,Café
4,North York,0.0,Gym / Fitness Center,Caribbean Restaurant,Basketball Court,Japanese Restaurant,Café,Construction & Landscaping,Diner,Event Space,Ethiopian Restaurant,Empanada Restaurant
5,North York,0.0,Park,Pizza Place,Metro Station,Pub,Japanese Restaurant,Discount Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
6,East York,0.0,Park,Cosmetics Shop,Beer Store,Bus Stop,Dance Studio,Skating Rink,Athletics & Sports,Asian Restaurant,Curling Ice,Video Store
7,Downtown Toronto,0.0,Coffee Shop,Café,Hotel,Restaurant,Clothing Store,Italian Restaurant,Bakery,Gastropub,Breakfast Spot,Cosmetics Shop
8,York,0.0,Field,Hockey Arena,Trail,Women's Store,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
9,East Toronto,0.0,Trail,Pub,Health Food Store,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
10,Downtown Toronto,0.0,Coffee Shop,Cocktail Bar,Farmers Market,Beer Bar,Steakhouse,Bakery,Café,Cheese Shop,Seafood Restaurant,Bistro
12,Scarborough,0.0,Coffee Shop,Korean Restaurant,Women's Store,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop


#### Cluster 2

In [34]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,1.0,Fast Food Restaurant,Park,Food & Drink Shop,Women's Store,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
11,York,1.0,Park,Women's Store,Fast Food Restaurant,Market,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
20,East York,1.0,Park,Convenience Store,Metro Station,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Women's Store
28,Central Toronto,1.0,Park,Lawyer,Bus Line,Swim School,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
30,York,1.0,Convenience Store,Park,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Women's Store
31,North York,1.0,Park,Bank,Bar,Convenience Store,Women's Store,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant
40,Downtown Toronto,1.0,Park,Trail,Playground,Building,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
63,North York,1.0,Park,Airport,Women's Store,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
77,Central Toronto,1.0,Park,Trail,Bus Line,Sushi Restaurant,Jewelry Store,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Deli / Bodega
87,North York,1.0,Basketball Court,Park,Bakery,Construction & Landscaping,Women's Store,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop


#### Cluster 3

In [35]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Scarborough,2.0,Playground,Women's Store,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
56,Scarborough,2.0,Park,Playground,Women's Store,Doner Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
67,Central Toronto,2.0,Gym,Playground,Women's Store,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant


#### Cluster 4

In [36]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
58,North York,3.0,Martial Arts Dojo,Women's Store,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore


#### Cluster 5

In [37]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
47,Scarborough,4.0,Bar,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore,Department Store


***

## 5.7 LONDON
### DATA RETRIEVAL

In [38]:
london_link='https://en.wikipedia.org/wiki/List_of_London_boroughs'

In [39]:
london_data= pd.read_html(london_link)
london_df= london_data[0]
london_df

,Borough,Inner,Status,Local authority,Political control,Headquarters,Area (sq mi),Population (2013 est)[1],Co-ordinates,Nr. in map
0,Barking and Dagenham [note 1],NaN,NaN,Barking and Dagenham London Borough Council,Labour,"Town Hall, 1 Town Square",13.93,194352,51°33′39″N 0°09′21″E﻿ / ﻿51.5607°N 0.1557°E,25
1,Barnet,NaN,NaN,Barnet London Borough Council,Conservative,"North London Business Park, Oakleigh Road South",33.49,369088,51°37′31″N 0°09′06″W﻿ / ﻿51.6252°N 0.1517°W,31
2,Bexley,NaN,NaN,Bexley London Borough Council,Conservative,"Civic Offices, 2 Watling Street",23.38,236687,51°27′18″N 0°09′02″E﻿ / ﻿51.4549°N 0.1505°E,23
3,Brent,NaN,NaN,Brent London Borough Council,Labour,"Brent Civic Centre, Engineers Way",16.70,317264,51°33′32″N 0°16′54″W﻿ / ﻿51.5588°N 0.2817°W,12
4,Bromley,NaN,NaN,Bromley London Borough Council,Conservative,"Civic Centre, Stockwell Close",57.97,317899,51°24′14″N 0°01′11″E﻿ / ﻿51.4039°N 0.0198°E,20
5,Camden,NaN,NaN,Camden London Borough Council,Labour,"Camden Town Hall, Judd Street",8.40,229719,51°31′44″N 0°07′32″W﻿ / ﻿51.5290°N 0.1255°W,11
6,Croydon,NaN,NaN,Croydon London Borough Council,Labour,"Bernard Weatherill House, Mint Walk",33.41,372752,51°22′17″N 0°05′52″W﻿ / ﻿51.3714°N 0.0977°W,19
7,Ealing,NaN,NaN,Ealing London Borough Council,Labour,"Perceval House, 14-16 Uxbridge Road",21.44,342494,51°30′47″N 0°18′32″W﻿ / ﻿51.5130°N 0.3089°W,13
8,Enfield,NaN,NaN,Enfield London Borough Council,Labour,"Civic Centre, Silver Street",31.74,320524,51°39′14″N 0°04′48″W﻿ / ﻿51.6538°N 0.0799°W,30
9,Greenwich [note 2],[note 3],Royal,Greenwich London Borough Council,Labour,"Woolwich Town Hall, Wellington Street",18.28,264008,51°29′21″N 0°03′53″E﻿ / ﻿51.4892°N 0.0648°E,22


In [40]:
# London West End covers the borough's of Westminster and Camden
# Let's extract them from london_df
choice= london_df.iloc[[5,31],:]
choice= choice[['Borough','Co-ordinates']]
choice

,Borough,Co-ordinates
5,Camden,51°31′44″N 0°07′32″W﻿ / ﻿51.5290°N 0.1255°W
31,Westminster,51°29′50″N 0°08′14″W﻿ / ﻿51.4973°N 0.1372°W


In [41]:
def derive_cood(x):
    choice['Longitude']= x.str.split('/').str.get(1).str.split('°').str.get(0)
    choice['Latitude']= x.str.split('/').str.get(1).str.split('°').str.get(1).str.split(' ').str.get(1)
    return choice

In [42]:
derive_cood(choice['Co-ordinates'])
westend_geo= choice[['Borough','Longitude','Latitude']]
westend_geo

,Borough,Longitude,Latitude
5,Camden,﻿51.5290,0.1255
31,Westminster,﻿51.4973,0.1372


In [43]:
london_pc= pd.read_csv(londonpc_url)
london_pc.head()

,Postcode,In Use?,Latitude,Longitude,Easting,Northing,Grid Ref,County,District,Ward,District Code,Ward Code,Country,County Code,Constituency,Introduced,Terminated,Parish,National Park,Population,Households,Built up area,Built up sub-division,Lower layer super output area,Rural/urban,Region,Altitude,London zone,LSOA Code,Local authority,MSOA Code,Middle layer super output area,Parish Code,Census output area,Constituency Code,Index of Multiple Deprivation,Quality,User Type,Last updated,Nearest station,Distance to station,Postcode area,Postcode district,Police force,Water company
0,BR1 1AA,Yes,51.401546,0.015415,540291,168873,TQ402688,Greater London,Bromley,Bromley Town,E09000006,E05000109,England,E11000009,Bromley and Chislehurst,2016-05-01,NaN,"Bromley, unparished area",NaN,NaN,NaN,Greater London,Bromley,Bromley 018B,Urban major conurbation,London,71,5,E01000675,NaN,E02000144,Bromley 018,E43000196,E00003264,E14000604,20532,1,0,2019-08-24,Bromley South,0.218257,BR,BR1,Metropolitan Police,Thames Water
1,BR1 1AB,Yes,51.406333,0.015208,540262,169405,TQ402694,Greater London,Bromley,Bromley Town,E09000006,E05000109,England,E11000009,Bromley and Chislehurst,2012-03-01,NaN,"Bromley, unparished area",NaN,NaN,NaN,Greater London,Bromley,Bromley 008B,Urban major conurbation,London,71,4,E01000676,NaN,E02000134,Bromley 008,E43000196,E00003255,E14000604,10169,1,0,2019-08-24,Bromley North,0.253666,BR,BR1,Metropolitan Police,Thames Water
2,BR1 1AD,No,51.400057,0.016715,540386,168710,TQ403687,Greater London,Bromley,Bromley Town,E09000006,E05000109,England,E11000009,Bromley and Chislehurst,2014-09-01,2017-09-01,"Bromley, unparished area",NaN,NaN,NaN,Greater London,Bromley,Bromley 018B,Urban major conurbation,London,53,5,E01000675,NaN,E02000144,Bromley 018,E43000196,E00003264,E14000604,20532,1,1,2019-08-24,Bromley South,0.044559,BR,BR1,Metropolitan Police,NaN
3,BR1 1AE,Yes,51.404543,0.014195,540197,169204,TQ401692,Greater London,Bromley,Bromley Town,E09000006,E05000109,England,E11000009,Bromley and Chislehurst,2008-08-01,NaN,"Bromley, unparished area",NaN,34.0,21.0,Greater London,Bromley,Bromley 018C,Urban major conurbation,London,71,4,E01000677,NaN,E02000144,Bromley 018,E43000196,E00003266,E14000604,19350,1,0,2019-08-24,Bromley North,0.462939,BR,BR1,Metropolitan Police,Thames Water
4,BR1 1AF,Yes,51.401392,0.014948,540259,168855,TQ402688,Greater London,Bromley,Bromley Town,E09000006,E05000109,England,E11000009,Bromley and Chislehurst,2015-05-01,NaN,"Bromley, unparished area",NaN,NaN,NaN,Greater London,Bromley,Bromley 018B,Urban major conurbation,London,58,5,E01000675,NaN,E02000144,Bromley 018,E43000196,E00003264,E14000604,20532,1,0,2019-08-24,Bromley South,0.227664,BR,BR1,Metropolitan Police,Thames Water


In [44]:
london_pc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320915 entries, 0 to 320914
Data columns (total 45 columns):
Postcode                          320915 non-null object
In Use?                           320915 non-null object
Latitude                          320915 non-null float64
Longitude                         320915 non-null float64
Easting                           320915 non-null int64
Northing                          320915 non-null int64
Grid Ref                          320915 non-null object
County                            320915 non-null object
District                          320915 non-null object
Ward                              320915 non-null object
District Code                     320915 non-null object
Ward Code                         320915 non-null object
Country                           320915 non-null object
County Code                       320915 non-null object
Constituency                      320915 non-null object
Introduced                        

In [45]:
col_lon=['Postcode','Latitude','Longitude','District','Ward','Postcode area']
london_pc1= london_pc.query('District == ["Westminster", "Camden"]')
london_pc1= london_pc1[col_lon]
print(london_pc1.shape)
london_pc1.head()

(47093, 6)


,Postcode,Latitude,Longitude,District,Ward,Postcode area
9793,CM23 3WE,51.528566,-0.116476,Camden,King's Cross,CM
56792,EC1M 3AB,51.521357,-0.106597,Camden,Holborn and Covent Garden,EC
56793,EC1M 3AD,51.521387,-0.106783,Camden,Holborn and Covent Garden,EC
56798,EC1M 3HA,51.519286,-0.105847,Camden,Holborn and Covent Garden,EC
56799,EC1M 3HB,51.518921,-0.106121,Camden,Holborn and Covent Garden,EC


In [46]:
london_pc1['Postalcode']= london_pc1.Postcode.str.split(' ', expand=True).get(0)
london_pc1.head()

,Postcode,Latitude,Longitude,District,Ward,Postcode area,Postalcode
9793,CM23 3WE,51.528566,-0.116476,Camden,King's Cross,CM,CM23
56792,EC1M 3AB,51.521357,-0.106597,Camden,Holborn and Covent Garden,EC,EC1M
56793,EC1M 3AD,51.521387,-0.106783,Camden,Holborn and Covent Garden,EC,EC1M
56798,EC1M 3HA,51.519286,-0.105847,Camden,Holborn and Covent Garden,EC,EC1M
56799,EC1M 3HB,51.518921,-0.106121,Camden,Holborn and Covent Garden,EC,EC1M


In [47]:
london_pc1.reset_index(inplace=True)

In [48]:
london_pc2= london_pc1[['Postalcode','Ward','District','Latitude','Longitude']]
print(london_pc2.shape)
london_pc2.head()

(47093, 5)


,Postalcode,Ward,District,Latitude,Longitude
0,CM23,King's Cross,Camden,51.528566,-0.116476
1,EC1M,Holborn and Covent Garden,Camden,51.521357,-0.106597
2,EC1M,Holborn and Covent Garden,Camden,51.521387,-0.106783
3,EC1M,Holborn and Covent Garden,Camden,51.519286,-0.105847
4,EC1M,Holborn and Covent Garden,Camden,51.518921,-0.106121


In [49]:
london_grp= london_pc2.groupby(['Postalcode','District'],as_index=False).agg({'Latitude':'max','Longitude':'max','Ward':'max'})
print(london_grp.shape)
london_grp.head()
#lambda x: ", ".join(x)

(97, 5)


,Postalcode,District,Latitude,Longitude,Ward
0,CM23,Camden,51.528566,-0.116476,King's Cross
1,EC1M,Camden,51.522004,-0.105847,Holborn and Covent Garden
2,EC1N,Camden,51.521950,-0.105688,Holborn and Covent Garden
3,EC1R,Camden,51.523507,-0.107306,Holborn and Covent Garden
4,EC4A,Camden,51.518092,-0.110480,Holborn and Covent Garden


In [50]:
london_grp.columns=['Postalcode','Borough','Latitude','Longitude','Neighborhood']
london_grp.head()

,Postalcode,Borough,Latitude,Longitude,Neighborhood
0,CM23,Camden,51.528566,-0.116476,King's Cross
1,EC1M,Camden,51.522004,-0.105847,Holborn and Covent Garden
2,EC1N,Camden,51.521950,-0.105688,Holborn and Covent Garden
3,EC1R,Camden,51.523507,-0.107306,Holborn and Covent Garden
4,EC4A,Camden,51.518092,-0.110480,Holborn and Covent Garden


## 5.8 LONDON
### DATA EXPLORATION

In [51]:
# Mapping Toronto
address = 'London, UK'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London, UK are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of London, UK are 51.5073219, -0.1276474.


In [52]:
# create map of Toronto using latitude and longitude values
map_london = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(london_grp['Latitude'], london_grp['Longitude'], london_grp['Borough'], london_grp['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

### <i> UTILIZING THE FOURSUARE API</i>

In [53]:
LIMIT=100 # setting limit returned by the Foursquare API

def getNearbyVenues2(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [54]:
london_venues = getNearbyVenues2(names=london_grp['Neighborhood'],
                                   latitudes=london_grp['Latitude'],
                                   longitudes=london_grp['Longitude']
                                  )

King's Cross
Holborn and Covent Garden
Holborn and Covent Garden
Holborn and Covent Garden
Holborn and Covent Garden
St James's
St James's
St James's
St Pancras and Somers Town
Kentish Town
St Pancras and Somers Town
Highgate
St Pancras and Somers Town
St Pancras and Somers Town
Regent's Park
Regent's Park
Fortune Green
Regent's Park
West Hampstead
Kentish Town
West Hampstead
Maida Vale
Swiss Cottage
Regent's Park
St James's
Vincent Square
Warwick
Vincent Square
Vincent Square
Warwick
Warwick
Vincent Square
Vincent Square
Knightsbridge and Belgravia
Vincent Square
Knightsbridge and Belgravia
Marylebone High Street
Queen's Park
Westbourne
Bloomsbury
West End
West End
West End
West End
West End
West End
West End
West End
West End
West End
Bloomsbury
West End
Bloomsbury
West End
West End
West End
Bloomsbury
West End
West End
West End
West End
West End
West End
Westbourne
Marylebone High Street
Kilburn
Westbourne
Holborn and Covent Garden
West End
Holborn and Covent Garden
West End
Holborn

In [55]:
#Shape of the resulting Dataframe
print(london_venues.shape)
london_venues.head()

(7882, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,King's Cross,51.528566,-0.116476,Gagosian Gallery,51.530125,-0.118065,Art Gallery
1,King's Cross,51.528566,-0.116476,Frequency,51.529110,-0.115797,Coffee Shop
2,King's Cross,51.528566,-0.116476,The Queen's Head,51.528343,-0.118558,Pub
3,King's Cross,51.528566,-0.116476,Aux Pains de Papy,51.529340,-0.120303,Bakery
4,King's Cross,51.528566,-0.116476,Sen Viet,51.529163,-0.115777,Vietnamese Restaurant


In [56]:
#How many Venues were retured for each Neighbourhood
london_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bloomsbury,400,400,400,400,400,400
Fortune Green,8,8,8,8,8,8
Highgate,28,28,28,28,28,28
Holborn and Covent Garden,1577,1577,1577,1577,1577,1577
Kentish Town,70,70,70,70,70,70
Kilburn,48,48,48,48,48,48
King's Cross,276,276,276,276,276,276
Knightsbridge and Belgravia,200,200,200,200,200,200
Maida Vale,37,37,37,37,37,37


In [57]:
#How many unique Venue categories?
print('There are {} uniques categories.'.format(len(london_venues['Venue Category'].unique())))

There are 276 uniques categories.


## 5.9 LONDON
### <i> NEIGHBOURHOOD ANALYSIS</i>

In [58]:
# one hot encoding
london_onehot = pd.get_dummies(london_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
london_onehot['Neighborhood'] = london_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [london_onehot.columns[-1]] + list(london_onehot.columns[:-1])
london_onehot = london_onehot[fixed_columns]

london_onehot.head()

,Zoo Exhibit,Accessories Store,African Restaurant,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Austrian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Field,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Stop,Business Service,Café,Camera Store,Canal,Candy Store,Casino,Caucasian Restaurant,Champagne Bar,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Residence Hall,Comedy Club,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Donut Shop,Dumpling Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Iraqi Restaurant,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kebab Restaurant,Korean Restaurant,Lake,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Massage Studio,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Okonomiyaki Restaurant,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoor Supply Store,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pharmacy,Piano Bar,Pie Shop,Pier,Piercing Parlor,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Smoke Shop,Snack Place,Soba Restaurant,Soccer Field,Social Club,Soup Place,Souvenir Shop,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stationery Store,Steakhouse,Street Food Gathering,Student Center,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Theme Restaurant,Tour Provider,Tourist Information Center,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Udon Restaurant,Used Bookstore,Vape Store,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,W

In [59]:
london_onehot.shape

(7882, 276)

In [60]:
london_grouped = london_onehot.groupby('Neighborhood').mean().reset_index()
print(london_grouped.shape)
london_grouped

(20, 276)


,Neighborhood,Zoo Exhibit,Accessories Store,African Restaurant,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Austrian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Field,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Stop,Business Service,Café,Camera Store,Canal,Candy Store,Casino,Caucasian Restaurant,Champagne Bar,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Residence Hall,Comedy Club,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Donut Shop,Dumpling Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Iraqi Restaurant,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kebab Restaurant,Korean Restaurant,Lake,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Massage Studio,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Okonomiyaki Restaurant,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoor Supply Store,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pharmacy,Piano Bar,Pie Shop,Pier,Piercing Parlor,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Smoke Shop,Snack Place,Soba Restaurant,Soccer Field,Social Club,Soup Place,Souvenir Shop,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stationery Store,Steakhouse,Street Food Gathering,Student Center,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Theme Restaurant,Tour Provider,Tourist Information Center,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Udon Restaurant,Used Bookstore,Vape Store,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,W

## 5.10 LONDON

### <i>TOP 5 VENUES</i>

In [61]:
num_top_venues = 5

for hood in london_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = london_grouped[london_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bloomsbury----
                  venue  freq
0           Coffee Shop  0.10
1                  Café  0.07
2                 Hotel  0.04
3  Gym / Fitness Center  0.04
4             Bookstore  0.03


----Fortune Green----
                 venue  freq
0        Grocery Store  0.25
1  Sporting Goods Shop  0.12
2          Coffee Shop  0.12
3    Indian Restaurant  0.12
4  Japanese Restaurant  0.12


----Highgate----
         venue  freq
0          Pub  0.21
1       Bakery  0.07
2  Coffee Shop  0.07
3         Café  0.07
4      Theater  0.04


----Holborn and Covent Garden----
            venue  freq
0     Coffee Shop  0.09
1             Pub  0.07
2           Hotel  0.04
3            Café  0.03
4  Sandwich Place  0.03


----Kentish Town----
                venue  freq
0       Grocery Store  0.11
1         Coffee Shop  0.11
2         Pizza Place  0.07
3            Bus Stop  0.06
4  Italian Restaurant  0.06


----Kilburn----
                  venue  freq
0                   Pub  0.08
1  Gym / 

In [62]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [63]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
london_venues_sorted = pd.DataFrame(columns=columns)
london_venues_sorted['Neighborhood'] = london_grouped['Neighborhood']

for ind in np.arange(london_grouped.shape[0]):
    london_venues_sorted.iloc[ind, 1:] = return_most_common_venues(london_grouped.iloc[ind, :], num_top_venues)

london_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bloomsbury,Coffee Shop,Café,Gym / Fitness Center,Hotel,Pub,Bookstore,Bar,Sandwich Place,Bakery,Pizza Place
1,Fortune Green,Grocery Store,Sporting Goods Shop,Indian Restaurant,Gym / Fitness Center,Japanese Restaurant,Other Great Outdoors,Coffee Shop,Film Studio,Fish & Chips Shop,Flea Market
2,Highgate,Pub,Coffee Shop,Bakery,Café,Jazz Club,French Restaurant,Theater,Tea Room,Grocery Store,Gastropub
3,Holborn and Covent Garden,Coffee Shop,Pub,Hotel,Sandwich Place,Café,French Restaurant,Italian Restaurant,Bookstore,Theater,Korean Restaurant
4,Kentish Town,Grocery Store,Coffee Shop,Pizza Place,Indian Restaurant,Bus Stop,Pub,Café,Italian Restaurant,Bar,Hotel


## 5.11 LONDON
### CLUSTERING

In [64]:
# set number of clusters
kclusters = 5

london_grouped_clustering = london_grouped.drop('Neighborhood', 1)

# run k-means clustering
lon_kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(london_grouped_clustering)

# check cluster labels generated for each row in the dataframe
lon_kmeans.labels_[0:10] 

array([0, 4, 1, 0, 0, 3, 3, 0, 3, 0], dtype=int32)

In [65]:
# add clustering labels
london_venues_sorted.insert(0, 'Cluster Labels', lon_kmeans.labels_)

london_merged = london_grp

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
london_merged = london_merged.join(london_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

london_merged.head()

,Postalcode,Borough,Latitude,Longitude,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,CM23,Camden,51.528566,-0.116476,King's Cross,3,Pub,Café,Coffee Shop,Hotel,Italian Restaurant,Vietnamese Restaurant,Cocktail Bar,Indian Restaurant,Garden,Pizza Place
1,EC1M,Camden,51.522004,-0.105847,Holborn and Covent Garden,0,Coffee Shop,Pub,Hotel,Sandwich Place,Café,French Restaurant,Italian Restaurant,Bookstore,Theater,Korean Restaurant
2,EC1N,Camden,51.521950,-0.105688,Holborn and Covent Garden,0,Coffee Shop,Pub,Hotel,Sandwich Place,Café,French Restaurant,Italian Restaurant,Bookstore,Theater,Korean Restaurant
3,EC1R,Camden,51.523507,-0.107306,Holborn and Covent Garden,0,Coffee Shop,Pub,Hotel,Sandwich Place,Café,French Restaurant,Italian Restaurant,Bookstore,Theater,Korean Restaurant
4,EC4A,Camden,51.518092,-0.110480,Holborn and Covent Garden,0,Coffee Shop,Pub,Hotel,Sandwich Place,Café,French Restaurant,Italian Restaurant,Bookstore,Theater,Korean Restaurant


In [66]:
london_merged.dropna(axis=0,inplace=True)
london_merged.head()

,Postalcode,Borough,Latitude,Longitude,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,CM23,Camden,51.528566,-0.116476,King's Cross,3,Pub,Café,Coffee Shop,Hotel,Italian Restaurant,Vietnamese Restaurant,Cocktail Bar,Indian Restaurant,Garden,Pizza Place
1,EC1M,Camden,51.522004,-0.105847,Holborn and Covent Garden,0,Coffee Shop,Pub,Hotel,Sandwich Place,Café,French Restaurant,Italian Restaurant,Bookstore,Theater,Korean Restaurant
2,EC1N,Camden,51.521950,-0.105688,Holborn and Covent Garden,0,Coffee Shop,Pub,Hotel,Sandwich Place,Café,French Restaurant,Italian Restaurant,Bookstore,Theater,Korean Restaurant
3,EC1R,Camden,51.523507,-0.107306,Holborn and Covent Garden,0,Coffee Shop,Pub,Hotel,Sandwich Place,Café,French Restaurant,Italian Restaurant,Bookstore,Theater,Korean Restaurant
4,EC4A,Camden,51.518092,-0.110480,Holborn and Covent Garden,0,Coffee Shop,Pub,Hotel,Sandwich Place,Café,French Restaurant,Italian Restaurant,Bookstore,Theater,Korean Restaurant


In [67]:
# create map
lon_map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(london_merged['Latitude'], london_merged['Longitude'], london_merged['Neighborhood'], london_merged['Cluster Labels'].astype(int)):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(lon_map_clusters)
       
lon_map_clusters

## 5.12 LONDON
### EXAMINING EACH CLUSTER

#### Cluster 1

In [68]:
london_merged.loc[london_merged['Cluster Labels'] == 0, london_merged.columns[[1] + list(range(5, london_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Camden,0,Coffee Shop,Pub,Hotel,Sandwich Place,Café,French Restaurant,Italian Restaurant,Bookstore,Theater,Korean Restaurant
2,Camden,0,Coffee Shop,Pub,Hotel,Sandwich Place,Café,French Restaurant,Italian Restaurant,Bookstore,Theater,Korean Restaurant
3,Camden,0,Coffee Shop,Pub,Hotel,Sandwich Place,Café,French Restaurant,Italian Restaurant,Bookstore,Theater,Korean Restaurant
4,Camden,0,Coffee Shop,Pub,Hotel,Sandwich Place,Café,French Restaurant,Italian Restaurant,Bookstore,Theater,Korean Restaurant
5,Westminster,0,Coffee Shop,Pub,Sandwich Place,Restaurant,Hotel,Italian Restaurant,Café,Theater,Wine Bar,Park
6,Westminster,0,Coffee Shop,Pub,Sandwich Place,Restaurant,Hotel,Italian Restaurant,Café,Theater,Wine Bar,Park
7,Westminster,0,Coffee Shop,Pub,Sandwich Place,Restaurant,Hotel,Italian Restaurant,Café,Theater,Wine Bar,Park
9,Camden,0,Grocery Store,Coffee Shop,Pizza Place,Indian Restaurant,Bus Stop,Pub,Café,Italian Restaurant,Bar,Hotel
14,Westminster,0,Pub,Coffee Shop,Music Venue,Beer Bar,Sushi Restaurant,Greek Restaurant,Burger Joint,Italian Restaurant,Japanese Restaurant,Café
15,Camden,0,Pub,Coffee Shop,Music Venue,Beer Bar,Sushi Restaurant,Greek Restaurant,Burger Joint,Italian Restaurant,Japanese Restaurant,Café


#### Cluster 2

In [69]:
london_merged.loc[london_merged['Cluster Labels'] == 1, london_merged.columns[[1] + list(range(5, london_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Camden,1,Pub,Coffee Shop,Bakery,Café,Jazz Club,French Restaurant,Theater,Tea Room,Grocery Store,Gastropub
18,Camden,1,Pub,Bakery,Coffee Shop,Pizza Place,Plaza,Gastropub,Lake,French Restaurant,Café,Indian Restaurant
20,Camden,1,Pub,Bakery,Coffee Shop,Pizza Place,Plaza,Gastropub,Lake,French Restaurant,Café,Indian Restaurant


#### Cluster 3

In [70]:
london_merged.loc[london_merged['Cluster Labels'] == 2, london_merged.columns[[1] + list(range(5, london_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,Westminster,2,Coffee Shop,Gym / Fitness Center,Gym,Italian Restaurant,Bar,Tapas Restaurant,Grocery Store,Farmers Market,Bakery,Park


#### Cluster 4

In [71]:
london_merged.loc[london_merged['Cluster Labels'] == 3, london_merged.columns[[1] + list(range(5, london_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Camden,3,Pub,Café,Coffee Shop,Hotel,Italian Restaurant,Vietnamese Restaurant,Cocktail Bar,Indian Restaurant,Garden,Pizza Place
8,Camden,3,Café,Grocery Store,Pub,Park,Brewery,Music Venue,Coffee Shop,Supermarket,Rental Car Location,Tennis Court
10,Camden,3,Café,Grocery Store,Pub,Park,Brewery,Music Venue,Coffee Shop,Supermarket,Rental Car Location,Tennis Court
12,Camden,3,Café,Grocery Store,Pub,Park,Brewery,Music Venue,Coffee Shop,Supermarket,Rental Car Location,Tennis Court
13,Camden,3,Café,Grocery Store,Pub,Park,Brewery,Music Venue,Coffee Shop,Supermarket,Rental Car Location,Tennis Court
21,Westminster,3,Café,Pub,Hotel,Italian Restaurant,French Restaurant,Middle Eastern Restaurant,Pizza Place,Gift Shop,Thai Restaurant,Gym / Fitness Center
22,Camden,3,Pub,Café,Coffee Shop,Italian Restaurant,Bar,French Restaurant,Bus Stop,Furniture / Home Store,Spa,Gourmet Shop
65,Camden,3,Pub,Hotel,Pizza Place,Café,Gym / Fitness Center,Bus Stop,Park,Middle Eastern Restaurant,Grocery Store,French Restaurant
73,Camden,3,Pub,Café,Coffee Shop,Hotel,Italian Restaurant,Vietnamese Restaurant,Cocktail Bar,Indian Restaurant,Garden,Pizza Place
75,Camden,3,Pub,Café,Coffee Shop,Hotel,Italian Restaurant,Vietnamese Restaurant,Cocktail Bar,Indian Restaurant,Garden,Pizza Place


#### Cluster 5

In [72]:
london_merged.loc[london_merged['Cluster Labels'] == 4, london_merged.columns[[1] + list(range(5, london_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,Camden,4,Grocery Store,Sporting Goods Shop,Indian Restaurant,Gym / Fitness Center,Japanese Restaurant,Other Great Outdoors,Coffee Shop,Film Studio,Fish & Chips Shop,Flea Market


# 6.0 RESULTS

The table below summarizes the first and second most popular venues in Toronto and London West End postal areas, based on the cluster analysis in the methodology section.

<table style="width:50%">
<tr>
<th>Cluster</th>
<th>Toronto</th>
<th>London West End</th>
</tr>

<tr>
<td>1</td>
<td>Park/Hockey Arena</td>
<td>Coffee Shop/Pub</td>
</tr>

<tr>
<td>2</td>
<td>Parks</td>
<td>Pub/Bakery</td>
</tr>

<tr>
<td>3</td>
<td>Playground/Park</td>
<td>Coffee Shop/Gym</td>
</tr>

<tr>
<td>4</td>
<td>Martial Arts/Women's Store</td>
<td>Cafe/Grocery Store</td>
</tr>

<tr>
<td>5</td>
<td>Bar</td>
<td>Grocery Store/Sporting Goods</td>
</tr>
</table>

# 7.0 DISCUSSION

The objectives set out at the Problem section of this notebook are:
 - How similar are the types of Venues in London West End compared to Toronto?
 - can we infer from the data that a NetherSpoons restaurant will be succesful
 - Identify the best 3 locations in Toronto that is similar to London's West End to site the restaurant

We will utilise the results from our cluster analysis to answer the problems. Firstly, both Toronto and London have the same number of venue categories at 276. However, the most common venues in Toronto and London are more dissimilar than similar. In Toronto, parks and playgrounds feature more, showing an outdoor style when it comes to the types of venues, whereas in London West End it is more indoor related. Coffee shops feature more prominently in London. Only Scarborough in Toronto has more bars and it's own cluster, the counterpart in London West End is larger, clustered with Bakery (Cluster 2). There are more dissimilarities in both locations.

Secondly, from the cluster analysis it may seem that because of the dissimilarity in the lifestyle choices of both cities, it will be difficult to see if NetherSpoons will be able to thrive. However, there are more restuarant's featuring in the first Toronto cluster than there are in London's first cluster. In most part's of London West End restaurants are the 6th most common, but in Toronto they are seen from the second most common venues. Presenting a good opportunity for NetherSpoon's global expansion.

Finally, The 3 best location for NetherSpoon's brand of restaurants will be North York, Downtown Toronto and East York. These are the areas with more restaurants with rich diversity, including Chinese and Italian restaurants.

# 8.0 CONCLUSION

The aim of this notebook was to determine the location and suitability of Toronto as the first North American expansion of NetherSpoon's brand of British restaurants. A subset of London, London West End was chosen for neighbourhood analysis and for comaprison with Toronto. Following data analysis of publically avaialble data, it can be determined that although Toronto and London West End are dissimilar in how people choose to spend their time; outdoors in Toronto and Indoors in London West End, however, when residents in Toronto choose to eat outside the home, they are more likely to do so in either North York, Downtown Toronto and East York. These neighbourhoods provides NetherSpoon's the greatest investment opportunity outside of London West End as they choose to expand globally.